In [1]:
import numpy as np
import pandas as pd
%matplotlib inline


import numpy as np

import pandas as pd
import numpy as np
import  seaborn as sns
import rwi as rwi
import math


In [117]:
def load_df(name):
    eurusd_df = pd.read_csv(name)
    eurusd_df = eurusd_df.rename(columns={'<TICKER>':'Ticker', '<DTYYYYMMDD>':'Date', '<TIME>':'Time', '<OPEN>':'Open', '<HIGH>':'High', '<LOW>':'Low', '<CLOSE>':'Close', '<VOL>':'Vol'})
    eurusd_df[['Date']] = pd.to_datetime(eurusd_df['Date'].astype('str'), yearfirst=True)
    eurusd_df =eurusd_df.sort_values(by=['Date', 'Time'])
    eur_df = eurusd_df[['Date', 'Time', 'Close']].rename(columns={'Close':'price'})
    eur_df['returns'] = eur_df['price'].pct_change()
    eur_df.dropna(inplace=True)
    eur_df.sort_values(by=['Date', 'Time'])
    eur_df = eur_df[eur_df.Date>'2019-01-01']
    return eur_df

In [3]:
instruments = ['eurusd', 'btc', 'spy']
tfs = ['1min', '5min', '15min', '60min', '240min', '1d', '1w', '1m']

In [ ]:

col_names = ['MomVal', 'MomDouble', 'h1', 'h2', 'h3', 'h4', 'MMIR', 'MMIP', 'ShannonVal', 'prop', 'var_std', 'var_mean', 'var_median', 'correl_1', 'correl_2', 'correl_3', 'correl_4', 'correl_5', 'correl_10', 'correl_20', 'correl_100']

In [62]:
if 0 in res1[4]:
    print(1)

1


In [75]:
eur_df

,Date,Time,price,returns
1,2003-05-06,1,1.12910,-0.000097
2,2003-05-06,2,1.12892,-0.000159
3,2003-05-06,3,1.12919,0.000239
4,2003-05-06,4,1.12919,0.000000
5,2003-05-06,5,1.12907,-0.000106
...,...,...,...,...
6097528,2020-02-14,2155,1.08302,-0.000018
6097529,2020-02-14,2156,1.08302,0.000000
6097530,2020-02-14,2157,1.08302,0.000000
6097531,2020-02-14,2158,1.08302,0.000000


In [82]:
tails_ratio(eur_df)

Date       0.0
Time       0.0
price      0.0
returns    0.0
dtype: float64

In [118]:
results = []
for inst in instruments:
    for tf in tfs:
        
        name = 'data/'+inst +'_'+tf+'.csv'
        
        eur_df = load_df(name)
        res1=rwi.generateRWI2(eur_df.reset_index())
        if type(res1[4])==list:
            res1[4] = res1[4][0]
            try:
                res1[5] = res5[5][0]
            except:
                res1[5] = 0
                continue
        row= pd.DataFrame(res1).T
        row.columns=col_names
        row['instrument'] = inst
        row['period'] = tf
        row['pain_to_gain'] = pain_to_gain_ratio(eur_df)
        row['tails_ratio'] = tails_ratio(eur_df)
        
        results.append(row.copy())

        

C:\Users\lexik\Documents\workspace\quant\ts_analysis\rwi.py:209: RuntimeWarning: divide by zero encountered in log
  m = np.polyfit(np.log(lags), np.log(tau), 1)


In [119]:
def pain_to_gain_ratio(df):
    
    a = df[df.returns > df.returns.std()*3].returns.count()
    b = df[df.returns < df.returns.std()*-3].returns.count()

    return a/b



In [120]:
def tails_ratio(df):
    
    a = df[(df.returns > df.returns.std()*3) | (df.returns < df.returns.std()*-3)]
    total = df.returns.count()
    
    return (a.returns.count()/total)*100



In [121]:
result_df = pd.concat(results,axis=0)

In [141]:
result_df.sort_values(by=['correl_20'])[['correl_20', 'instrument', 'period']]

,correl_20,instrument,period
0,-0.107739,eurusd,1d
0,-0.032794,btc,240min
0,-0.015807,btc,1d
0,-0.014944,spy,1d
0,-0.013197,spy,240min
0,-0.013112,eurusd,240min
0,-0.009713,eurusd,15min
0,-0.004391,btc,1min
0,-0.001811,eurusd,5min
0,-0.001541,spy,15min


In [123]:
result_df.to_pickle('rwi measures 2019_01_01.pkl')

In [114]:
result_df.corr()

,MomVal,MomDouble,h1,h2,h3,h4,MMIR,MMIP,ShannonVal,prop,...,correl_1,correl_2,correl_3,correl_4,correl_5,correl_10,correl_20,correl_100,pain_to_gain,tails_ratio
MomVal,1.000000,0.993723,0.011067,-0.134475,0.060039,0.220391,-0.186861,0.053990,-0.224531,0.301654,...,0.000246,0.257848,-0.133030,0.052962,-0.101623,-0.087892,0.168736,-0.101410,-0.103425,0.051281
MomDouble,0.993723,1.000000,0.011838,-0.048649,-0.004931,0.253877,-0.212526,-0.006262,-0.194092,0.267781,...,-0.031084,0.204662,-0.231322,0.142392,-0.088772,-0.115684,0.124248,-0.195531,-0.098716,0.072295
h1,0.011067,0.011838,1.000000,0.385677,0.565340,0.168016,0.291874,0.737465,0.407260,-0.374997,...,0.793443,0.186363,0.100870,-0.052655,0.560842,-0.098683,-0.418784,0.175766,0.667499,0.625321
h2,-0.134475,-0.048649,0.385677,1.000000,-0.349840,0.040715,-0.087765,-0.153189,0.716582,-0.696231,...,0.044899,-0.478271,-0.513868,0.615659,0.582152,-0.221478,-0.799016,-0.548316,0.625779,0.357081
h3,0.060039,-0.004931,0.565340,-0.349840,1.000000,0.311101,0.207816,0.868092,-0.117651,0.194861,...,0.554875,0.582606,0.581878,-0.565314,0.327355,0.185712,0.212694,0.661599,0.594224,0.259983
h4,0.220391,0.253877,0.168016,0.040715,0.311101,1.000000,0.110631,0.107480,-0.325481,0.490207,...,0.075521,-0.028965,-0.405850,0.232753,0.026852,-0.393108,0.257622,-0.348379,-0.143834,0.433310
MMIR,-0.186861,-0.212526,0.291874,-0.087765,0.207816,0.110631,1.000000,0.460375,-0.212738,0.261802,...,0.551180,-0.155190,-0.016117,0.011185,-0.167051,-0.000216,0.255343,0.059228,-0.285737,0.546956
MMIP,0.053990,-0.006262,0.737465,-0.153189,0.868092,0.107480,0.460375,1.000000,0.122263,0.007154,...,0.676363,0.546373,0.501595,-0.501579,0.427561,0.120817,-0.002823,0.659245,0.558685,0.332216
ShannonVal,-0.224531,-0.194092,0.407260,0.716582,-0.117651,-0.325481,-0.212738,0.122263,1.000000,-0.879133,...,0.051748,-0.126029,0.050352,0.010462,0.759081,-0.186531,-0.927687,0.046165,0.617984,-0.033683
prop,0.301654,0.267781,-0.374997,-0.696231,0.194861,0.490207,0.261802,0.007154,-0.879133,1.000000,...,-0.143665,0.173502,-0.075315,-0.021248,-0.583527,-0.141343,0.828364,0.006061,-0.684492,-0.058770
